In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random

from tensorflow import keras
from keras import layers
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3

In [2]:
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      os.path.join(dirname, filename)

In [3]:
train_dir = 'Training/Training'
test_dir = 'Testing/Testing'

train_angry_dir = 'Training/Training/Angry'
train_fear_dir = 'Training/Training/Fear'
train_happy_dir = 'Training/Training/Happy'
train_neutral_dir = 'Training/Training/Neutral'
train_sad_dir = 'Training/Training/Sad'
train_surprise_dir = 'Training/Training/Suprise'

test_angry_dir = 'Testing/Testing/Angry'
test_fear_dir = 'Testing/Testing/Fear'
test_happy_dir = 'Testing/Testing/Happy'
test_neural_dir = 'Testing/Testing/Neutral'
test_sad_dir = 'Testing/Testing/Sad'
test_surprise_dir = 'Testing/Testing/Suprise'

In [4]:
dir_list = [train_angry_dir, train_fear_dir, train_happy_dir,
           train_neutral_dir, train_sad_dir, train_surprise_dir,
           test_angry_dir, test_fear_dir, test_happy_dir,
           test_neural_dir, test_sad_dir, test_surprise_dir]

In [5]:
for d in dir_list:
    print(d, len(os.listdir(d)))


Training/Training/Angry 3995
Training/Training/Fear 4097
Training/Training/Happy 7215
Training/Training/Neutral 4965
Training/Training/Sad 4830
Training/Training/Suprise 3171
Testing/Testing/Angry 958
Testing/Testing/Fear 1024
Testing/Testing/Happy 1774
Testing/Testing/Neutral 1233
Testing/Testing/Sad 1247
Testing/Testing/Suprise 831


In [6]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150),
                                                   batch_size=64,
                                                   class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

validation_generator = validation_datagen.flow_from_directory(test_dir,
                                                             target_size=(150,150),
                                                             batch_size=62,
                                                             class_mode='categorical')

Found 28273 images belonging to 6 classes.
Found 7067 images belonging to 6 classes.


In [7]:
# callback function

early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,
                                                 restore_best_weights=True)

In [8]:
# build the model

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(6, activation='softmax')  
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

model.summary()
# keras.utils.plot_model(model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        3

In [9]:
# train the model

history = model.fit(train_generator,
                   epochs=10,
                   verbose=1,
                   validation_data=validation_generator,
                   callbacks=early_stopping_cb)

Epoch 1/10
 53/442 [==>...........................] - ETA: 3:41 - loss: 1.7736 - accuracy: 0.2541

KeyboardInterrupt: 

In [ ]:
model.save('emotion_detect.h5')

In [ ]:
model_new = tf.keras.models.load_model ("emotion_detect.h5")